# Исследование надежности заемщиков. 

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных


In [1]:
import pandas as pd # импортирую библиотеку pandas
try:
    df = pd.read_csv('/datasets/data.csv') #загружаю данные 
except: 
    df = pd.read_csv('data.csv')
df.info() 
df.head() #знакомлюсь с данными

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.1 Заполнение пропусков

После вызова df.info() видно, что общее количество значений в таблице 21525. В двух столбцах df['total_income'] и df['days_employed'] количество  значений 19351, значит в этих столбцах есть пропуски. Доля пропущенных значений составляет 10%.

In [2]:
print ('Доля пропущенных значений составляет:',round((len(df[df['total_income'].isna()])) * 100 / len(df['total_income']))) # Доля пропущенных значений составляет 10%.

Доля пропущенных значений составляет: 10


In [3]:
df['days_employed'] = df['days_employed'].abs() #поскольку в столбце df['days_employed'] есть отрицательные значения, применяю метод abs()
df['days_employed'] = df['days_employed'].fillna(value=df['days_employed'].median()) # заполняю пропускии в столбце df['days_employed'] медианным значением
df['total_income'] = df['total_income'].fillna(value=df['total_income'].median()) # тоже самое для столбца df['total_income']
df.info() # пропусков больше нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Поскольку данные могут содержать аномалии, то пропуски в количественных переменных лучше заполнять медианным значением, так данные будут более корректны. 
Причины появления пропусков могут быть различны. Приведу приведу самые очевидные:
- ошибка оператора, который заполнял данные;
- ошибка при выгрузке данных.


### Шаг 2.2 Проверка данных на аномалии и исправления.

В процессе обработки данных встретились значения, которые скорее всего не отражают действительность и появились по какой-то ошибке. Например:
- возраст 0 лет в столбце df['	dob_years'];
- отрицательное значение трудового стажа в столбце df['days_employed'];
- отрицательное значение количества детей в столбце df['children'];
- большое значение количества детей в столбце df['children'].
Поскольку параметр "Возраст заёмщика", обозначенный в столбце df['	dob_years'] в исследовании не участвует, то оставим его без внимания.
С отрицательным значением трудового стажа мы разобрались в шаге 2.1, приведя значения в положительные, использовоав метод abs().
Полскольку отрицательное значение количества детей в столбце df['children'] я считаю опечаткой, то приведём это значение к положительному методом abs().
20 детей вероятнее всего быть не может, тк это слишком сильно выбивается из средних показателей. Скорее всего детей 2. Для исправления этой аномалии использую метод индексмации с оператором loc.


In [4]:
df['children'] = df['children'].abs() # привожу отрицательное значение к положительному.
df.loc[df['children'] == 20 , 'children'] = 2 # заменяю значение "20" на значение "2"
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.

Для изменения вещественного типа данных в столбце df['total_income'] на целочисленный выполню с помощью метода astype():

In [5]:
df['total_income'] = df['total_income'].astype('int') #Изменяю тип данных в столбце df['total_income'] методом astype()
df.info() # проверяю: тип данных был "float64", стал "int64" 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Для проверки явных дубликатов применю метод duplicated() с функцией sum() чтобы посмотреть количество возможных дубликатов:

In [6]:
df.duplicated().sum()

54

Всего 54 дубля из 21525 строк, очень немного. Посмотрю на эти строки:

In [7]:
df[df.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194.220567,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,2194.220567,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,2194.220567,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,2194.220567,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,2194.220567,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
9238,2,2194.220567,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9528,0,2194.220567,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9627,0,2194.220567,56,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции со своей недвижимостью
10462,0,2194.220567,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,покупка коммерческой недвижимости


Метод принял за дубликаты медианные значения по столбцам df['total_income'] и df['days_employed']. Однако же остальные данные таблицы отличаются. Соответственно дубликатами такие строки считать недопустимо. 

Медотом value_counts() проверим таблицу на наличие неявных дубликатов:

In [8]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

D столбце df['education'] есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Привожу их к отдному регистру используя метод str.lower():

In [9]:
df['education'] = df['education'].str.lower()
df['education'].value_counts() #проверяю, изменились ли значения. Теперь они записаны одинаково. 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [10]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце df['purpose'] встречаются значения с одинаковым смыслом, но записанные по разному. Эти значения смело можно признать дубликатами.
Поскольку я уже устранил бардак в данных и создал новые категории в этом шаге до того, как дошёл до шага 2.7, объединяю это задание с шагом 2.7 "Категоризации целей кредита". Для этого создам столбец df[purpose_category], операции буду проводить для него.
Избавляюсь от дубликатов вызвав метод replace(), а чтобы не повторять его много много раз создам специальную собственную функцию, отдельно для каждой категории:

In [11]:
df['purpose_category'] = df['purpose']

Начну со значения "свадьба" одной из самых неблагодарных процедур на планете. У меня было три свадьбы, и ниодна не окупилась. Никому не порекомендую тратить на это событие много денег, особенно, брать на него кредит. Так что если ты, дорогой мой проверяющий, в ближайшем будущем планируешь свадьбу, то прими мой совет: трать на это событие как можно меньше денег (Минутка юмора на АналитикFM). 

In [12]:
def replace_wrong_values(wrong_values, correct_value): # на вход функции подаются список неправильных значений и строка с правильным значением
    for wrong_value in wrong_values: # перебираем неправильные значения
        df['purpose_category'] = df['purpose_category'].replace(wrong_value, correct_value) # и для каждого неправильного значения вызываем метод replace()

duplicates = ['на проведение свадьбы', 'сыграть свадьбу', 'свадьба'] # список неправильных значений
name = 'проведение свадьбы' # правильное значение
replace_wrong_values(duplicates, name) # вызов функции, replace() внутри будет вызван нужное количество раз
df['purpose_category'].value_counts() # проверяю изменения, неявные дубликаты устранены

проведение свадьбы                        2348
операции с недвижимостью                   676
покупка коммерческой недвижимости          664
операции с жильем                          653
покупка жилья для сдачи                    653
операции с коммерческой недвижимостью      651
покупка жилья                              647
жилье                                      647
покупка жилья для семьи                    641
строительство собственной недвижимости     635
недвижимость                               634
операции со своей недвижимостью            630
строительство жилой недвижимости           626
покупка недвижимости                       624
покупка своего жилья                       620
строительство недвижимости                 620
ремонт жилью                               612
покупка жилой недвижимости                 607
на покупку своего автомобиля               505
заняться высшим образованием               496
автомобиль                                 495
сделка с поде

Продолжу со значением "операции с недвижимостью". В таблице есть разделение на коммерческую и жилую. Так же есть значение просто недвижимость, без указания назначения. Поэтому я объединяю коммерческую, жилую и любуые иные процедуры с недвижимостью в одну категорию "операции с недвижимостью":

In [13]:
def replace_wrong_values(wrong_values, correct_value): # на вход функции подаются список неправильных значений и строка с правильным значением
    for wrong_value in wrong_values: # перебираем неправильные значения
        df['purpose_category'] = df['purpose_category'].replace(wrong_value, correct_value) # и для каждого неправильного значения вызываем метод replace()

duplicates = [
    'покупка коммерческой недвижимости',
    'покупка жилья для сдачи',
    'операции с жильем',
    'операции с коммерческой недвижимостью',
    'жилье',
    'покупка жилья',
    'покупка жилья для семьи',
    'строительство собственной недвижимости',
    'недвижимость',
    'операции со своей недвижимостью',
    'строительство жилой недвижимости',
    'покупка недвижимости',
    'покупка своего жилья',
    'строительство недвижимости',
    'ремонт жилью',
    'покупка жилой недвижимости'
    
             ] # список неправильных значений, мать его
name = 'операции с недвижимостью' # правильное значение
replace_wrong_values(duplicates, name) # вызов функции, replace() внутри будет вызван нужное количество раз
df['purpose_category'].value_counts() # проверяю изменения, бардак в недвижимости уничтожен

операции с недвижимостью                 10840
проведение свадьбы                        2348
на покупку своего автомобиля               505
заняться высшим образованием               496
автомобиль                                 495
сделка с подержанным автомобилем           489
свой автомобиль                            480
на покупку подержанного автомобиля         479
автомобили                                 478
на покупку автомобиля                      472
приобретение автомобиля                    462
дополнительное образование                 462
сделка с автомобилем                       455
высшее образование                         453
получение дополнительного образования      447
образование                                447
получение образования                      443
профильное образование                     436
получение высшего образования              426
заняться образованием                      412
Name: purpose_category, dtype: int64

Аналогичным образом поступаем со значением "автомобиль":

In [14]:
def replace_wrong_values(wrong_values, correct_value): # на вход функции подаются список неправильных значений и строка с правильным значением
    for wrong_value in wrong_values: # перебираем неправильные значения
        df['purpose_category'] = df['purpose_category'].replace(wrong_value, correct_value) # и для каждого неправильного значения вызываем метод replace()

duplicates = [
    'на покупку своего автомобиля',
    'сделка с подержанным автомобилем',
    'свой автомобиль',
    'на покупку подержанного автомобиля',
    'автомобили',
    'на покупку автомобиля',
    'приобретение автомобиля',
    'сделка с автомобилем',
    'автомобиль'
             ] # список неправильных значений
name = 'операции с автомобилем' # правильное значение
replace_wrong_values(duplicates, name) # вызов функции, replace() внутри будет вызван нужное количество раз
df['purpose_category'].value_counts() # проверяю изменения, автомобили на местах, навёл порядок в гараже

операции с недвижимостью                 10840
операции с автомобилем                    4315
проведение свадьбы                        2348
заняться высшим образованием               496
дополнительное образование                 462
высшее образование                         453
получение дополнительного образования      447
образование                                447
получение образования                      443
профильное образование                     436
получение высшего образования              426
заняться образованием                      412
Name: purpose_category, dtype: int64

И последнюю очередь (но ни в коем случае ни по важности) работаем со значением "образование". 

In [15]:
def replace_wrong_values(wrong_values, correct_value): # на вход функции подаются список неправильных значений и строка с правильным значением
    for wrong_value in wrong_values: # перебираем неправильные значения
        df['purpose_category'] = df['purpose_category'].replace(wrong_value, correct_value) # и для каждого неправильного значения вызываем метод replace()

duplicates = [
    'заняться высшим образованием',
    'дополнительное образование',
    'высшее образование',
    'получение дополнительного образования',
    'образование',
    'профильное образование',
    'получение высшего образования',
    'заняться образованием'    
             ] # список неправильных значений
name = 'получение образования' # правильное значение
replace_wrong_values(duplicates, name) # вызов функции, replace() внутри будет вызван нужное количество раз
df['purpose_category'].value_counts() # проверяю изменения, конец неявным дублиуатам, позорящим наши датафреймы

операции с недвижимостью    10840
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

In [16]:
df.head(20) #с превиликим удовольствием смотрю на таблицу с новым столюцом df['purpose_category'] и значениями в нём

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью


Когда неявные дубликаты устранены, воцарила красота и эстетика, можно порассуждать о причинах их дубликатов. 
В столбце df['education'] вероятнее всего причина техническая, дубликаты могли возникнуть при форматировании или обработке данных.
В столбце df['purpose'] вероятнее всего причины иные, а именно:
- разгильдяйство: в организации не определён единый эталон записи данных. А поскольку строк с данными 21525, то верорятно данные заполняли разные сотрудники. Соответственно, каждый сотрудник записывал так, как ему больше нравится, что могло привести к появлению дубликатов;
- диверсия: кто-то умышленно изменил данные. Например, человек, который готовил данные для Анализа мог умышленно "подпортить" значения. Мог он сделать это из-за чувств личной неприязни или же из хулиганских побуждений. 
- комплексная причина: комбинация вышеописанных причин в разной пропорции.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаю два новых датафрейма:

In [17]:
df_education = df[['education', 'education_id']] #рад бы в комментариях что-то умное написать, но писать тут нечего
df_education.head()

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


In [18]:
df_family_status = df[['family_status', 'family_status_id']] # такая же история, как и пунктом выше
df_family_status.head()

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Удаляю из исходного датафрейма столбцы df['education'] и df['family_status'] применяя метод pop():

In [19]:
df.pop('education')
df.pop('family_status')
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы


### Шаг 2.6. Категоризация дохода.

Для категоризации уровня дохода заёмщиков, создаю новый столбец df[total_income_category]:

In [20]:
def income_category (row): #создаю функцию, которая возвращает категорию, в зависимости от дохода.
    total_income = row['total_income']
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'
df['total_income_group'] = df.apply(income_category, axis = 1) #создаю столбец, используя метод apply(). 
print (df['total_income_group'].value_counts())
df.head(10) 

C    16087
B     5041
D      350
A       25
E       22
Name: total_income_group, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_group
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,получение образования,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью,C


### Шаг 2.7. Категоризация целей кредита.

Выполнил в шаге 2.4. 

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того, чтобы ответ на этот вопрос было проще дать, создам сводную таблицу df_children методом pivot_table():

In [21]:
df_children = df.pivot_table(
    index = ['children'],
    columns = 'debt',
    values = 'dob_years',
    aggfunc = 'count') #создаю новую таблицу, использую перенос для наглядности и удобства проверки. 
df_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6 non-null      float64
 1   1       5 non-null      float64
dtypes: float64(2)
memory usage: 144.0 bytes


In [22]:
df_children.columns = ['Нет задолженности', 'Есть задолженность'] #присваиваю столбцам новые значения
df_children = df_children.fillna(0) #появился пропуск, заполняю его
df_children['Нет задолженности'] = df_children['Нет задолженности'].astype('int')# видимо при создании df_children значениям присвоился тип "float64", меняю его на int
df_children['Есть задолженность'] = df_children['Есть задолженность'].astype('int')# аналогично предыдущему пункту
df_children.info() #проверял типы данных и отсутствие пропусков

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Нет задолженности   6 non-null      int64
 1   Есть задолженность  6 non-null      int64
dtypes: int64(2)
memory usage: 144.0 bytes


In [23]:
df_children['Доля возврата'] = (
    df_children['Нет задолженности'] / 
    (df_children['Нет задолженности'] + df_children['Есть задолженность'])
                                ) #создаю новый столбец, в котором считаю долю возврата
df_children['Процент возврата'] = df_children['Доля возврата'] * 100 # создаю столбец, в котором считаю процент возврата
df_children # любуюсь замечательной красивой таблией и говорящими данными в ней

,Нет задолженности,Есть задолженность,Доля возврата,Процент возврата
children,,,,
0,13086,1063,0.924871,92.487102
1,4420,445,0.908530,90.853032
2,1929,202,0.905209,90.520882
3,303,27,0.918182,91.818182
4,37,4,0.902439,90.243902
5,9,0,1.000000,100.000000


In [24]:
df_children['Процент возврата'].max() - df_children['Процент возврата'].min()
# считаю разницу между ссамыми надёжными и ненадёжными заёмщиками

9.756097560975604

##### Вывод 1:  
У людей, которые не имеют детей процент возврата кредитов выше, хотя не на много, на 2,24%. Однако, семьи, имеющие 5 детей вовсе не имеют задолженности. Но количество семей с пятью детьми несравнимо мало по отношению к общей массе заёмщиков. 
Зависимость между количеством детей и возвратом кредита в срок есть, но она незначительна.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос, применяю такой же метод, как и в предыдущем задании:

In [25]:
df_status_family = df.pivot_table(
    index = ['family_status_id'],
    columns = 'debt',
    values = 'dob_years',
    aggfunc = 'count') #создаю новую таблицу, использую перенос для наглядности и удобства проверки. 
df_status_family.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       5 non-null      int64
 1   1       5 non-null      int64
dtypes: int64(2)
memory usage: 120.0 bytes


In [26]:
df_status_family.columns = ['Нет задолженности', 'Есть задолженность'] #присваиваю столбцам новые значения
#пропусков тут не образовалось, тип данных "int64", шаг игнорирую
df_status_family.info() #проверял типы данных и отсутствие пропусков

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Нет задолженности   5 non-null      int64
 1   Есть задолженность  5 non-null      int64
dtypes: int64(2)
memory usage: 120.0 bytes


In [27]:
df_status_family['Доля возврата'] = (
    df_status_family['Нет задолженности'] / 
    (df_status_family['Нет задолженности'] + df_status_family['Есть задолженность'])
                                ) #создаю новый столбец, в котором считаю долю возврата
df_status_family['Процент возврата'] = df_status_family['Доля возврата'] * 100 # создаю столбец, в котором считаю процент возврата
df_status_family # любуюсь замечательной красивой таблией и говорящими данными в ней

,Нет задолженности,Есть задолженность,Доля возврата,Процент возврата
family_status_id,,,,
0,11449,931,0.924798,92.479806
1,3789,388,0.907110,90.711037
2,897,63,0.934375,93.437500
3,1110,85,0.928870,92.887029
4,2539,274,0.902595,90.259509


In [28]:
df_status_family['Процент возврата'].max() - df_status_family['Процент возврата'].min() 
# считаю разницу между ссамыми надёжными и ненадёжными заёмщиками

3.1779905794525405

##### Вывод 2:

Лучше всего кредиты отдают вдовцы, хуже всего не женатые и живущие гражданским браком. Хотя максимальный разброс по самым лучшим заёмщикам и самым худшим составляет всего 3,18 %, что немного. Поэтому, зависимость между семейным положением и возвратом кредита в срок есть, но она незначительная.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Цели другие, методы те же. Применяю такой же метод, как и п предыдущем задании.

In [29]:
df_education_id = df.pivot_table(
    index = ['education_id'],
    columns = 'debt',
    values = 'dob_years',
    aggfunc = 'count') #создаю новую таблицу, использую перенос для наглядности и удобства проверки. 
df_education_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5 non-null      float64
 1   1       4 non-null      float64
dtypes: float64(2)
memory usage: 120.0 bytes


In [30]:
df_education_id.columns = ['Нет задолженности', 'Есть задолженность'] #присваиваю столбцам новые значения
df_education_id = df_education_id.fillna(0) #появился пропуск, заполняю его
df_education_id['Нет задолженности'] = df_education_id['Нет задолженности'].astype('int')# видимо при создании df_education_id значениям присвоился тип "float64", меняю его на int
df_education_id['Есть задолженность'] = df_education_id['Есть задолженность'].astype('int')# аналогично предыдущему пункту
df_education_id.info() #проверял типы данных и отсутствие пропусков

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Нет задолженности   5 non-null      int64
 1   Есть задолженность  5 non-null      int64
dtypes: int64(2)
memory usage: 120.0 bytes


In [31]:
df_education_id['Доля возврата'] = (
    df_education_id['Нет задолженности'] / 
    (df_education_id['Нет задолженности'] + df_education_id['Есть задолженность'])
                                ) #создаю новый столбец, в котором считаю долю возврата
df_education_id['Процент возврата'] = df_education_id['Доля возврата'] * 100 # создаю столбец, в котором считаю процент возврата
df_education_id # любуюсь замечательной красивой таблией и говорящими данными в ней

,Нет задолженности,Есть задолженность,Доля возврата,Процент возврата
education_id,,,,
0,4982,278,0.947148,94.714829
1,13869,1364,0.910458,91.045756
2,676,68,0.908602,90.860215
3,251,31,0.890071,89.007092
4,6,0,1.000000,100.000000


In [32]:
df_education_id['Процент возврата'].max() - df_education_id['Процент возврата'].min() 
# считаю разницу между ссамыми надёжными и ненадёжными заёмщиками

10.99290780141844

##### Вывод 3: 
Из таблицы видно, что 100% дают заёмщики с учёной степенью. Да, их мало, но это крайне надёжные заёмщики. Ненадёжными заёмщиками являются люди с начальным образованием (хотя их тоже немного, относительно общего числа заёмщиков). В остальных случаях разница невелика, и составляет 3.9%. Зависимость между уровнем дохода и возвратом кредита в срок есть.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Применяю такой же метод, как и в заданиях выше:

In [33]:
df_purpose = df.pivot_table(
    index = ['purpose_category'],
    columns = 'debt',
    values = 'dob_years',
    aggfunc = 'count') #создаю новую таблицу, использую перенос для наглядности и удобства проверки. 
df_purpose.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, операции с автомобилем to проведение свадьбы
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       4 non-null      int64
 1   1       4 non-null      int64
dtypes: int64(2)
memory usage: 96.0+ bytes


In [34]:
df_purpose.columns = ['Нет задолженности', 'Есть задолженность'] #присваиваю столбцам новые значения
#пропусков тут не образовалось, тип данных "int64", шаг игнорирую
df_purpose.info() #проверял типы данных и отсутствие пропусков

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, операции с автомобилем to проведение свадьбы
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Нет задолженности   4 non-null      int64
 1   Есть задолженность  4 non-null      int64
dtypes: int64(2)
memory usage: 96.0+ bytes


In [35]:
df_purpose['Доля возврата'] = (
    df_purpose['Нет задолженности'] / 
    (df_purpose['Нет задолженности'] + df_purpose['Есть задолженность'])
                                ) #создаю новый столбец, в котором считаю долю возврата
df_purpose['Процент возврата'] = df_purpose['Доля возврата'] * 100 # создаю столбец, в котором считаю процент возврата
df_purpose # любуюсь замечательной красивой таблией и говорящими данными в ней

,Нет задолженности,Есть задолженность,Доля возврата,Процент возврата
purpose_category,,,,
операции с автомобилем,3912,403,0.906605,90.660487
операции с недвижимостью,10058,782,0.927860,92.785978
получение образования,3652,370,0.908006,90.800597
проведение свадьбы,2162,186,0.920784,92.078365


In [36]:
df_purpose['Процент возврата'].max() - df_purpose['Процент возврата'].min() 
# считаю разницу между ссамыми надёжными и ненадёжными заёмщиками

2.1254911853869345

##### Вывод 4:
Для меня удивительно, но по категории "проведение свадьбы" процент возврата один из самых больших. Так же удивительно, что "получение образования" один из самых низких процентов возврата. Я думал, что будет наоборот.
Хуже всего возвращают кредиты на "операции с автомобилями", что и не удивительно: автомобили разбивают, ломают, перепродают, и платить за них уже не хочется. Лидером по возврату является категория "операции с недвижимостью". 


## Общий вывод:

В процессе выполнения проекта и ответов на вопросы обнаружены следующие закономерности:
1. Заёмщики, у  которых нет детей, отдают кредиты в срок чаще, чем те, у кого дети есть. 
2. Заёмщики, которые состоят или состояли в официальном браке лучше отдают кредиты, чем те заёмщики, которе в браке никогда не состояли.
3. Присутствует чёткая корреляция между процентом по возврату кредита в срок и уровнем образования у заёмщиков. Чем выше уровень образования, тем чаще возвращают кредиты вовремя.
4. Кредиты, взятые на операции с недвижимостью и на проведение свадьбы, заёмщики отдают в срок чаще, чем люди, которые оформляют кредиты на операции с автомобилем и на получение образования.